In [13]:
import numpy as np
import pandas as pd

from scipy import interpolate
from scipy import integrate

In [14]:
dNdLogx_df = pd.read_csv('../..//DataFiles//PP-Factor/AtProduction_gammas.dat', delim_whitespace=True)

In [15]:
ebins = 2*np.logspace(-1,3,41)

In [28]:
dNdLogx_df

,mDM,"Log[10,x]",eL,eR,e,\[Mu]L,\[Mu]R,\[Mu],\[Tau]L,\[Tau]R,...,Z,g,\[Gamma],h,\[Nu]e,\[Nu]\[Mu],\[Nu]\[Tau],V->e,V->\[Mu],V->\[Tau]
0,5,-8.90,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,5,-8.85,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,5,-8.80,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,5,-8.75,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,-8.70,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,5,-8.65,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,5,-8.60,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,5,-8.55,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,5,-8.50,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,5,-8.45,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
min(dNdLogx_df['mDM'])

5

In [31]:
m_ary = [5,10,20]
channel='q'

PPnoxsec_ary = np.zeros(shape=(len(m_ary),len(ebins)-1))

for mi in range(len(m_ary)):

    dNdLogx_ann_df = dNdLogx_df.query('mDM == ' + (str(np.int(float(m_ary[mi])))))[['Log[10,x]',channel]]
    Egamma = np.array(m_ary[mi]*(10**dNdLogx_ann_df['Log[10,x]']))
    dNdEgamma = np.array(dNdLogx_ann_df[channel]/(Egamma*np.log(10)))
    dNdE_interp = interpolate.interp1d(Egamma, dNdEgamma)
    for ei in range(len(ebins)-1): # -1 because self.ebins-1 bins, self.ebins edges
        if ebins[ei] < m_ary[mi]: # Only have flux if m > Ebin
            if ebins[ei+1] < m_ary[mi]: # Whole bin is inside
                PPnoxsec_ary[mi,ei] = 1.0/(8*np.pi*m_ary[mi]**2)*integrate.quad(lambda x: dNdE_interp(x), ebins[ei], ebins[ei+1])[0]
            else: # Bin only partially contained
                PPnoxsec_ary[mi,ei] = 1.0/(8*np.pi*m_ary[mi]**2)*integrate.quad(lambda x: dNdE_interp(x), ebins[ei], m_ary[mi])[0]

In [32]:
PPnoxsec_ary

array([[  1.04781998e-03,   1.05503595e-03,   1.01602870e-03,
          9.32431441e-04,   8.08626431e-04,   6.62149933e-04,
          5.11034187e-04,   3.68780029e-04,   2.47512548e-04,
          1.52185798e-04,   8.26543759e-05,   3.84744962e-05,
          1.40157628e-05,   3.68828438e-06,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  3.35416567e-04,   3.45824865e-04,   3.43636011e-04,
          3.28641970e-04,   3.01133667e-04,   2.65259450e-04,
          2.24959108e-04,   1.83128392e-04,